# dbachecks PR check on Beard NUC

Rob uses this process to make sure that changes will work in an AD estate with a variety of versions

First we must check the status of the local development folder

In [1]:
cd D:\OneDrive\Documents\GitHub\dbachecks
# git checkout development
git status

On branch development
Your branch is ahead of 'origin/development' by 13 commits.
  (use "git push" to publish your local commits)

nothing to commit, working tree clean


First we need to check out the PR locally. To do this go to Github and open the PR and at the bottom you will see

![command line instructions](.\Images\commandline.png)

Click on that and you will see the code that you need

![command line details](.\Images\commandlinedetails.png)


In [2]:
git checkout -b tboggiano-serviceaccountadmins development
git pull https://github.com/tboggiano/dbachecks.git serviceaccountadmins

M	PSScriptAnalyzerSettings.psd1
Switched to a new branch 'tboggiano-serviceaccountadmins'
error: refs/remotes/tboggiano/serviceaccountadmins does not point to a valid object!
error: refs/remotes/tboggiano/serviceaccountadmins does not point to a valid object!
error: refs/remotes/tboggiano/serviceaccountadmins does not point to a valid object!
From https://github.com/tboggiano/dbachecks
 * branch            serviceaccountadmins -> FETCH_HEAD
error: Your local changes to the following files would be overwritten by merge:
	PSScriptAnalyzerSettings.psd1
Please commit your changes or stash them before you merge.
Aborting


Sometimes there are merge conflicts - these need to be resolved - i will do this i n Visual Studio Code

In [10]:
git merge development

Merge made by the 'recursive' strategy.
 PSScriptAnalyzerSettings.psd1                     |   1 +
 RELEASE.md                                        |  20 +-
 checks/Agent.Tests.ps1                            |  41 ++--
 checks/Database.Tests.ps1                         |  62 ++++++
 checks/HADR.Tests.ps1                             |  22 +-
 checks/Instance.Tests.ps1                         | 113 +++++++++-
 dbachecks.psd1                                    |  16 +-
 docs/RELEASE.md                                   |  10 +-
 docs/functions/Invoke-DbcCheck.md                 |  86 +++++++-
 internal/assertions/Database.Assertions.ps1       |  24 +++
 internal/assertions/Instance.Assertions.ps1       | 243 +++++++++++++++++++++-
 internal/configurations/DbcCheckDescriptions.json |  37 +++-
 internal/configurations/configuration.ps1         |  20 +-
 tests/Unit.Tests.ps1                              |   2 +-
 14 files changed, 634 insertions(+), 63 deletions(-)


First we need to import the module from source control - I normally use verbose to see how it is going


In [2]:
Import-Module .\dbachecks.psd1 -Verbose
$null = Reset-DbcConfig

VERBOSE: Loading module from path 'D:\OneDrive\Documents\GitHub\dbachecks\dbachecks.psd1'.
VERBOSE: Loading 'FormatsToProcess' from path 'D:\OneDrive\Documents\GitHub\dbachecks\xml\dbachecks.Format.ps1xml'.
VERBOSE: Loading module from path 'D:\OneDrive\Documents\GitHub\dbachecks\dbachecks.psm1'.
VERBOSE: Exporting function 'Import-ModuleFile'.
VERBOSE: Exporting function 'Get-CheckFile'.
VERBOSE: Exporting function 'Get-CheckInformation'.
VERBOSE: Exporting function 'Get-CheckRepo'.
VERBOSE: Exporting function 'Get-ComputerName'.
VERBOSE: Exporting function 'Get-DatabaseDetail'.
VERBOSE: Exporting function 'Get-Instance'.
VERBOSE: Exporting function 'Get-Version'.
VERBOSE: Exporting function 'Invoke-ConfigurationScript'.
VERBOSE: Exporting function 'New-Json'.
VERBOSE: Exporting function 'Select-DefaultView'.
VERBOSE: Exporting function 'Set-DatabaseForIntegrationTesting'.
VERBOSE: Exporting function 'Clear-DbcPowerBiDataSource'.
VERBOSE: Exporting function 'Export-DbcConfig'.
VERBOSE

Excellent, that worked. Now to run a single check

In [3]:
Invoke-DbcCheck -SqlInstance SQL2017N5 -Check AutoClose

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1' with Tags AutoClose

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Database.Tests.ps1

  Describing Auto Close

    Context Testing Auto Close on SQL2017N5
      [+] ADonutFactory on SQL2017N5 should have Auto Close set to False 151ms
      [+] ASausageFactory on SQL2017N5 should have Auto Close set to False 12ms
      [+] ASockFactory on SQL2017N5 should have Auto Close set to False 10ms
      [+] master on SQL2017N5 should have Auto Close set to False 13ms
      [+] model on SQL2017N5 should have Auto Close set to False 11ms
      [+] msdb on SQL2017N5 should have Auto Close set to False 14ms
      [+] SqlWatch on SQL2017N5 should have Auto Close set to False 11ms
      [+] SSISDB on SQL2017N5 should have Auto Close s

Run a single check with various Shows

In [4]:
Invoke-DbcCheck -SqlInstance SQL2017N5 -Check FailedJob -Show Fails

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Agent.Tests.ps1' with Tags FailedJob

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Agent.Tests.ps1

  Describing Failed Jobs

    Context Checking for failed enabled jobs since 02/20/2020 15:59:26 on SQL2017N5
      [-] DatabaseBackup - USER_DATABASES - FULL's last run outcome on SQL2017N5 is Failed 13ms
        Expected 'Succeeded', because All Agent Jobs should have succeed - you need to investigate the failed jobs, but got Failed.
        169:                                         $psitem.LastRunOutcome | Should -Be "Succeeded" -Because 'All Agent Jobs should have succeed - you need to investigate the failed jobs'
        at <ScriptBlock>, D:\OneDrive\Documents\GitHub\dbachecks\checks\Agent.Tests.ps1: line 169
      [-] Gather SQLWat

In [6]:
Invoke-DbcCheck -SqlInstance SQL2017N5 -Check BuiltInAdmin

    ____            __
   / __ \___  _____/ /____  _____
  / /_/ / _ \/ ___/ __/ _ \/ ___/
 / ____/  __(__  ) /_/  __/ /
/_/    \___/____/\__/\___/_/
Pester v4.9.0
Executing all tests in 'D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1' with Tags BuiltInAdmin

Executing script D:\OneDrive\Documents\GitHub\dbachecks\checks\Instance.Tests.ps1

  Describing Login BUILTIN Administrators cannot exist

    Context Checking that a login named BUILTIN\Administrators does not exist on SQL2017N5
      [!] BUILTIN\Administrators login does not exist on SQL2017N5, is skipped 0ms
Tests completed in 440ms
Tests Passed: 0, Failed: 0, Skipped: 1, Pending: 0, Inconclusive: 0 
